In [1]:
import pandas as pd

# Function to process the combined_data_1 file
def read_netflix_ratings(file_path):
    data = []  # List to store each rating as a list
    current_movie_id = None  # Variable to keep track of the current movie ID
    with open(file_path, 'r') as file:
        for line in file:
            if line.endswith(':\n'):  # This line is a movie ID
                current_movie_id = line[:-2]  # Remove the colon and newline to get the ID
            else:  # This line is a rating
                customer_id, rating, date = line.strip().split(',')
                data.append([current_movie_id, customer_id, rating, date])
    return pd.DataFrame(data, columns=['MovieID', 'CustomerID', 'Rating', 'Date'])

# Path to the combined_data_1 file
ratings_file_path = 'C:/Users/nafla/OneDrive/Documents/system development/Netflix/combined_data_1.txt'
ratings_df = read_netflix_ratings(ratings_file_path)


In [5]:
ratings_df.head()

,MovieID,CustomerID,Rating,Date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


In [7]:
def read_movie_titles(file_path):
    data = []  # Initialize an empty list to store the data
    with open(file_path, 'r', encoding='latin1') as file:
        for line in file:
            parts = line.strip().split(',', 2)  # Split each line, expecting at least 2 parts
            if len(parts) >= 2:  # Check if we have at least two parts (MovieID and Year)
                movie_id, year_of_release = parts[:2]  # Extract MovieID and YearOfRelease
                data.append([movie_id, year_of_release])
            else:
                print(f"Skipping malformed line: {line.strip()}")
    # Convert the list to a DataFrame with only MovieID and YearOfRelease
    return pd.DataFrame(data, columns=['MovieID', 'YearOfRelease'])

movie_titles_path = 'C:/Users/nafla/OneDrive/Documents/system development/Netflix/movie_titles.csv'
movies_df = read_movie_titles(movie_titles_path)

# Convert MovieID and YearOfRelease to the appropriate data types
movies_df['MovieID'] = movies_df['MovieID'].astype(int)
# Handle cases where YearOfRelease might be missing or malformed
movies_df['YearOfRelease'] = pd.to_numeric(movies_df['YearOfRelease'], errors='coerce')

# Display the first few rows of the DataFrame
movies_df.head()


,MovieID,YearOfRelease
0,1,2003.0
1,2,2004.0
2,3,1997.0
3,4,1994.0
4,5,2004.0


In [11]:
# Ensure that MovieID is the correct data type in both DataFrames
ratings_df['MovieID'] = ratings_df['MovieID'].astype(int)
movies_df['MovieID'] = movies_df['MovieID'].astype(int)

# Create a dictionary from movies_df mapping MovieID to YearOfRelease
year_of_release_dict = movies_df.set_index('MovieID')['YearOfRelease'].to_dict()

# Use the map function to create a new column in ratings_df for YearOfRelease
ratings_df['YearOfRelease'] = ratings_df['MovieID'].map(year_of_release_dict)

# Display the first few rows to verify the new column has been added correctly
ratings_df.head()


,MovieID,CustomerID,Rating,Date,YearOfRelease
0,1,1488844,3,2005-09-06,2003.0
1,1,822109,5,2005-05-13,2003.0
2,1,885013,4,2005-10-19,2003.0
3,1,30878,4,2005-12-26,2003.0
4,1,823519,3,2004-05-03,2003.0


In [15]:
# Print out the data types and counts
ratings_df.info()

# Calculate the percentage of missing values for each column
missing_percentage = ratings_df.isnull().mean() * 100

# Display the missing percentages
print(missing_percentage)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24053764 entries, 0 to 24053763
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   MovieID        int32  
 1   CustomerID     object 
 2   Rating         object 
 3   Date           object 
 4   YearOfRelease  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 825.8+ MB
MovieID          0.000000
CustomerID       0.000000
Rating           0.000000
Date             0.000000
YearOfRelease    0.000786
dtype: float64


In [16]:
# Check for duplicate rows
duplicate_rows = ratings_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 0


In [17]:
# Calculate the number of unique values in each column
unique_values_count = ratings_df.nunique()

# Display the number of unique values for each column
print(unique_values_count)

MovieID            4499
CustomerID       470758
Rating                5
Date               2182
YearOfRelease        89
dtype: int64


In [21]:

# Change the CustomerID data type as integer
ratings_df['CustomerID'] = ratings_df['CustomerID'].astype(int)

# Change the Rating data type as integer
ratings_df['Rating'] = ratings_df['Rating'].astype(int)

# Change the Date data type as date time
ratings_df['Date'] = pd.to_datetime(ratings_df['Date'], format='%Y-%m-%d')

# Change the YearOfRelease data type to object (string)
ratings_df['YearOfRelease'] = ratings_df['YearOfRelease'].astype(str)


In [23]:
# Replace null values in the YearOfRelease column with "Nan"
ratings_df['YearOfRelease'] = ratings_df['YearOfRelease'].fillna("Nan")

ratings_df.info()
# Calculate the percentage of missing values for each column
missing_percentage = ratings_df.isnull().mean() * 100
# Display the missing percentages
print(missing_percentage)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24053764 entries, 0 to 24053763
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   MovieID        int32         
 1   CustomerID     int32         
 2   Rating         int32         
 3   Date           datetime64[ns]
 4   YearOfRelease  object        
dtypes: datetime64[ns](1), int32(3), object(1)
memory usage: 642.3+ MB
MovieID          0.0
CustomerID       0.0
Rating           0.0
Date             0.0
YearOfRelease    0.0
dtype: float64


In [24]:
ratings_df.head()

,MovieID,CustomerID,Rating,Date,YearOfRelease
0,1,1488844,3,2005-09-06,2003.0
1,1,822109,5,2005-05-13,2003.0
2,1,885013,4,2005-10-19,2003.0
3,1,30878,4,2005-12-26,2003.0
4,1,823519,3,2004-05-03,2003.0
